In [3]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import time, os

from datetime import datetime, timedelta
from itertools import repeat

import dateutil.parser

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import ElementClickInterceptedException, StaleElementReferenceException, NoSuchWindowException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import re
import json

import pandas as pd
import numpy as np

In [4]:
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)

In [ ]:
driver.get('https://www.tripadvisor.com.sg/Hotels-g294265-Singapore-Hotels.html')
soup = BeautifulSoup(driver.page_source, "html.parser")

In [ ]:
columns = ['hotel_name', 'rating', 'walk_score', 'res_score', 'attraction_score', 'num_reviews2', 'num_QA', 'num_Tips', 'price_range', 'num_rooms']
df = pd.DataFrame([], columns = columns)

In [ ]:
list_of_hotels = []

all_hotels_on_page = driver.find_elements_by_xpath('//div[@class="listing_title"]//a')  

try:
    hotel_name = soup.find('h1', id="HEADING").text        
    print('hotel_name:', hotel_name)
    list_of_hotels.append(hotel_name)
except:
    pass

try:
    rating = soup.find('span', class_='hotels-hotel-review-about-with-photos-Reviews__overallRating--vElGA').contents[0]
    pass
except:
    list_of_hotels.append('NA')
else:
    list_of_hotels.append(rating)


try: 
    walk_score = re.findall("^[0-9]+", soup.find_all('span', class_='hotels-hotel-review-location-layout-Highlight__number--S3wsZ hotels-hotel-review-location-layout-Highlight__green--3lccI')[0].text)
    print('walk_score:', walk_score)
except:
    list_of_hotels.append('NA')
else:
    list_of_hotels.append(walk_score)
    
try:
    res_score = re.findall("^[0-9]+", soup.find_all('span', class_='hotels-hotel-review-location-layout-Highlight__number--S3wsZ hotels-hotel-review-location-layout-Highlight__orange--1N-BP')[0].text)
except:
    list_of_hotels.append('NA')
else:
    list_of_hotels.append(res_score)

try:
    attraction_score = re.findall("^[0-9]+", soup.find_all('span', class_='hotels-hotel-review-location-layout-Highlight__number--S3wsZ hotels-hotel-review-location-layout-Highlight__blue--2qc3K')[0].text)
except:
    list_of_hotels.append('NA')
else:
    list_of_hotels.append(attraction_score)


com_con = soup.find_all('span', class_='hotels-community-content-common-TabAboveHeader__tabCount--26Tct')
try: 
    num_reviews = com_con[0].text
except:
    list_of_hotels.append('NA')
else:
    list_of_hotels.append(num_reviews)

try:
    num_QA = com_con[1].text
except:
    list_of_hotels.append('NA')
else:
    list_of_hotels.append(num_QA)

try:
    num_Tips = com_con[2].text
except:
    list_of_hotels.append('NA')
else:
    list_of_hotels.append(num_Tips)


try:
    price_range = (str(soup.find('div', class_='hotels-hotel-review-about-addendum-AddendumItem__content--iVts5').text)).split('(')[0]
    
except:
    list_of_hotels.append('NA')
else:
    list_of_hotels.append(price_range)
    
allfeatures = soup.find_all('div', class_='ssr-init-26f hotels-hotel-review-layout-Section__plain--3fYKb')
for div in allfeatures:
    num_rooms = (div.find_all('div')[-1].text)            
list_of_hotels.append(num_rooms)

time.sleep(1)


In [ ]:
#save to dataframe 
df = df.append(pd.DataFrame(np.array(list_of_hotels).reshape(-1,len(list_of_hotels)), columns = columns), ignore_index=True)

driver.switch_to.window(driver.window_handles[0])

df.to_csv('TAhotels.csv', encoding='utf-8', index=False)